In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)

In [3]:
N_EPOCHS = 25
N_TRIALS = 50
BATCH_SIZE = 512

CONFIG = config_tests["assessment_proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [4]:
x, y = utils.load_cifar_data("../data/cifar-10-batches-py/")

In [5]:
x /= 255

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.6, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [ ]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

assessment_model.fit(x_train, y_train, epochs=25, batch_size=BATCH_SIZE)

Epoch 1/25


2024-03-23 16:57:08.788768: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-03-23 16:57:08.788794: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-03-23 16:57:08.788800: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-03-23 16:57:08.788820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-23 16:57:08.788833: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-03-23 16:57:09.708097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.1171 - loss: 2.4632
Epoch 2/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.1818 - loss: 2.2408
Epoch 3/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.2118 - loss: 2.1368
Epoch 4/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.2382 - loss: 2.0613
Epoch 5/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.2642 - loss: 1.9944
Epoch 6/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.2892 - loss: 1.9333
Epoch 7/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.2960 - loss: 1.8961
Epoch 8/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.3049 - loss: 1.8596
Epoch 9/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.3107 - loss: 1.8238
Epoch 10/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.3312 - loss: 1.7898
Epoch 11/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.3324 - loss: 1.7758
Epoch 12/25
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.3425 - l

In [7]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

samples_proba = utils.calculate_proba(
    assessment_model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
)

In [8]:
results = {}
# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]


for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=n_classes, **models_hyperparameters[test_model])
        model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)

            samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])

            model.fit(
                x_train_sorted[samples_ids],
                y_train_sorted[samples_ids],
                # validation_data=(x_val, y_val),
                epochs=1,
                batch_size=BATCH_SIZE,
                verbose=0,
            )
            
            if CONFIG["progressive"]:
                samples_proba = utils.calculate_proba(
                    model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
                )

        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)

        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/50 [00:00<?, ?it/s]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6873 - loss: 0.9240


  2%|▏         | 1/50 [01:21<1:06:35, 81.54s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6190 - loss: 1.1069


...

 96%|█████████▌| 48/50 [5:16:37<13:24, 402.31s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6742 - loss: 0.9389


 98%|█████████▊| 49/50 [5:23:13<06:40, 400.30s/it]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6813 - loss: 0.9067


100%|██████████| 50/50 [5:29:59<00:00, 396.00s/it]


In [9]:
results

{'test_model_1': [0.6743999719619751,
  0.6011999845504761,
  0.5415999889373779,
  0.6237999796867371,
  0.6615999937057495,
  0.6761999726295471,
  0.6462000012397766,
  0.5699999928474426,
  0.6205999851226807,
  0.5968000292778015,
  0.604200005531311,
  0.5860000252723694,
  0.6636000275611877,
  0.6452000141143799,
  0.5587999820709229,
  0.6496000289916992,
  0.631600022315979,
  0.6492000222206116,
  0.6462000012397766,
  0.6173999905586243,
  0.6868000030517578,
  0.574400007724762,
  0.5649999976158142,
  0.5410000085830688,
  0.6669999957084656,
  0.6218000054359436,
  0.6453999876976013,
  0.5839999914169312,
  0.5432000160217285,
  0.6614000201225281,
  0.6758000254631042,
  0.63919997215271,
  0.6452000141143799,
  0.6366000175476074,
  0.6517999768257141,
  0.5902000069618225,
  0.6470000147819519,
  0.4657999873161316,
  0.6711999773979187,
  0.6340000033378601,
  0.6733999848365784,
  0.656000018119812,
  0.6208000183105469,
  0.6407999992370605,
  0.61080002784729,
  

In [10]:
sum(results["test_model_1"]) / N_TRIALS

0.6244360017776489